In [75]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import skeletonDefMixamo as skd
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

from PhaseFunctionedNetwork import PhaseFunctionedNetwork

# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# load data
# X = np.float32(np.loadtxt('./data/Input.txt'))
# Y = np.float32(np.loadtxt('./data/Output.txt'))
# P = np.float32(np.loadtxt('./data/Phases.txt'))
# print(X.shape, Y.shape, P.shape)

In [5]:
def get_phases(X_input_arr):
    print(X_input_arr.shape[0]/2)
    # phase = np.linspace(0, 2*np.pi, num=X_input_arr.shape[0])
    # return phase

In [6]:
# def get_phases(n_elements):
#     # Determine the size of each segment (one-third of the total elements)
#     segment_size = n_elements // 3
    
#     # Create arrays for each segment
#     first_segment = np.zeros(segment_size)
#     second_segment = np.full(segment_size, np.pi)
#     third_segment = np.full(segment_size, 2 * np.pi)
    
#     # Concatenate the segments to form the final array
#     custom_array = np.concatenate((first_segment, second_segment, third_segment))
    
#     # If n_elements is not exactly divisible by 3, pad the array to reach the desired size
#     if custom_array.size < n_elements:
#         pad_size = n_elements - custom_array.size
#         custom_array = np.pad(custom_array, (0, pad_size), 'constant', constant_values=(2 * np.pi,))
    
#     return custom_array

In [89]:
# Process Data (Note currently this data is NOT MIRRORD, CAN DOUBLE WITH MIRRORING)

# Idle Data
X_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Idle_MIRROR.txt'))
Y_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Idle_MIRROR.txt'))
# phase hack
idle_phase_1 = np.linspace(0, 2*np.pi, num=995)
idle_phase_2 = np.linspace(0, 2*np.pi, num=995)
P_idle = np.concatenate((idle_phase_1, idle_phase_2))
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}")

# # Big Forward Jump
X_jump_for_BIG = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_jump_for_BIG_MIRROR.txt'))
Y_jump_for_BIG = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_jump_for_BIG_MIRROR.txt'))
# phase hack
jump_for_BIG_phase_1 = np.linspace(0, 2*np.pi, num=169)
jump_for_BIG_phase_2 = np.linspace(0, 2*np.pi, num=169)
P_jump_for_BIG = np.concatenate((jump_for_BIG_phase_1, jump_for_BIG_phase_2))
print(f"Jump Forward BIG data shape: {X_jump_for_BIG.shape}, {Y_jump_for_BIG.shape}, {P_jump_for_BIG.shape}")

# Small Forward Jump
X_jump_for_SMALL = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_jump_for_SMALL_MIRROR.txt'))
Y_jump_for_SMALL = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_jump_for_SMALL_MIRROR.txt'))
# phase hack
jump_for_SMALL_phase_1 = np.linspace(0, 2*np.pi, num=112)
jump_for_SMALL_phase_2 = np.linspace(0, 2*np.pi, num=112)
P_jump_for_SMALL = np.concatenate((jump_for_SMALL_phase_1, jump_for_SMALL_phase_2))
print(f"Jump Forward SMALL data shape: {X_jump_for_SMALL.shape}, {Y_jump_for_SMALL.shape}, {P_jump_for_SMALL.shape}")

# Big Up Jump
X_jump_up_BIG = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_jump_up_BIG_MIRROR.txt'))
Y_jump_up_BIG = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_jump_up_BIG_MIRROR.txt'))
# phase hack
jump_up_BIG_phase_1 = np.linspace(0, 2*np.pi, num=79)
jump_up_BIG_phase_2 = np.linspace(0, 2*np.pi, num=79)
P_jump_up_BIG = np.concatenate((jump_up_BIG_phase_1, jump_up_BIG_phase_2))
print(f"Jump Up BIG data shape: {X_jump_up_BIG.shape}, {Y_jump_up_BIG.shape}, {P_jump_up_BIG.shape}")

# Small Up Jump
X_jump_up_SMALL = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_jump_up_SMALL_MIRROR.txt'))
Y_jump_up_SMALL = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_jump_up_SMALL_MIRROR.txt'))
# phase hack
jump_up_SMALL_phase_1 = np.linspace(0, 2*np.pi, num=79)
jump_up_SMALL_phase_2 = np.linspace(0, 2*np.pi, num=79)
P_jump_up_SMALL = np.concatenate((jump_up_SMALL_phase_1, jump_up_SMALL_phase_2))
print(f"Jump Up SMALL data shape: {X_jump_up_SMALL.shape}, {Y_jump_up_SMALL.shape}, {P_jump_up_SMALL.shape}")

Idle data shape: (1990, 888), (1990, 819), (1990,)
Jump Forward BIG data shape: (338, 888), (338, 819), (338,)
Jump Forward SMALL data shape: (224, 888), (224, 819), (224,)
Jump Up BIG data shape: (158, 888), (158, 819), (158,)
Jump Up SMALL data shape: (158, 888), (158, 819), (158,)


In [90]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_idle, X_jump_for_BIG, X_jump_for_SMALL, X_jump_up_BIG, X_jump_up_SMALL))
print(f"X: {X.shape}")

Y = np.concatenate((Y_idle, Y_jump_for_BIG, Y_jump_for_SMALL, Y_jump_up_BIG, Y_jump_up_SMALL))
print(f"Y: {Y.shape}")

P = np.concatenate((P_idle, P_jump_for_BIG, P_jump_for_SMALL, P_jump_up_BIG, P_jump_up_SMALL))
print(f"P: {P.shape}")

X: (2868, 888)
Y: (2868, 819)
P: (2868,)


In [96]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_idle, X_jump_for_BIG, X_jump_for_SMALL))
print(f"X: {X.shape}")

Y = np.concatenate((Y_idle, Y_jump_for_BIG, Y_jump_for_SMALL))
print(f"Y: {Y.shape}")

P = np.concatenate((P_idle, P_jump_for_BIG, P_jump_for_SMALL))
print(f"P: {P.shape}")

X: (2552, 888)
Y: (2552, 819)
P: (2552,)


In [97]:
# Phase logic
delta_phase = P[1:] - P[:-1]
delta_phase[delta_phase < 0] = (1.0 - P[:-1] + P[1:])[delta_phase < 0]
delta_phase = np.append(delta_phase, 2 * np.pi)

Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

# calculate mean and std
Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1

# normalize data
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

# save mean / std / min / max

Xmean.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Demo/Dev/Weights/test/Xmean.bin')
Ymean.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Demo/Dev/Weights/test/Ymean.bin')
Xstd.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Demo/Dev/Weights/test/Xstd.bin')
Ystd.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Demo/Dev/Weights/test/Ystd.bin')

In [98]:
# append phase as additional feature
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

BATCH_SIZE = 32
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

Input shape torch.Size([2552, 889])
Target shape torch.Size([2552, 820])


In [81]:
# ensure GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
device = "cuda"

# custom loss function
def loss_func(output, target, model):
    loss = torch.mean((output - target)**2) + model.cost()
    return loss

In [99]:
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

epochs=3

for epoch in range(epochs):
    model.train()
    loss_list = []
    for i, batch in enumerate(tqdm(train_dataloader)):
        input, target = batch
        input, target = input.to(device), target.to(device)

        # forward pass
        output = model(input)
        loss = loss_func(output, target, model)
        loss_list.append(loss.item())

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {np.average(loss_list)}')


100%|██████████| 80/80 [00:07<00:00, 11.20it/s]


Epoch [1/3], Loss: 0.9233235165065563


100%|██████████| 80/80 [00:06<00:00, 11.48it/s]


Epoch [2/3], Loss: 0.7177801566682515


100%|██████████| 80/80 [00:06<00:00, 11.51it/s]

Epoch [3/3], Loss: 0.6550676562431244


In [100]:
# save weights
model.precompute_and_save_weights()